### LLM: pretrain

In [3]:
%pip install transformers torch datasets tokenizers accelerate numpy torch trl tqdm matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.4 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 1:
# Импорты

import os
import re
import json
from collections import Counter
from typing import List, Dict, Any
import torch
from torch.utils.data import Dataset
import numpy as np
from transformers import (
    PreTrainedTokenizer, 
    PreTrainedTokenizerFast,
    AutoTokenizer,
    LlamaConfig,
    LlamaForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback,
    GenerationConfig
)
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import tqdm as notebook_tqdm

print("Импорты выполнены ✅")

Импорты выполнены ✅


In [4]:
# 2: 
# Конфигурация подпроекта

class Config:
    DATA_DIR = "./data"  # здесь датасет - .txt файлы
    VOCAB_SIZE = 3000
    CONTEXT_LENGTH = 512
    MODEL_SAVE_PATH = "./llm_model"
    TOKENIZER_SAVE_PATH = "./tokenizer"
    
    # Параметры модели
    HIDDEN_SIZE = 1024
    INTERMEDIATE_SIZE = 1536
    NUM_HIDDEN_LAYERS = 16
    NUM_ATTENTION_HEADS = 16
    NUM_KEY_VALUE_HEADS = 8
    
    # Тестовые промпты
    TEST_PROMPTS = [
        "Все мысли, которые имеют огромные последствия",
        "Сила войска зависит от его духа",
        "Мысль о том, что он принес страдания",
        "Человек сознает себя свободным",
        "Что бы ни случилось, я всегда буду",
        "Любовь мешает смерти",
        "Нет, жизнь не кончена",
        "Всякая мысль, даже самая простая",
        "Война не любезность, а самое гадкое дело",
        "Чтобы жить честно"
    ]

# Создание необходимых папок
os.makedirs(Config.DATA_DIR, exist_ok=True)
os.makedirs(Config.MODEL_SAVE_PATH, exist_ok=True)
os.makedirs(Config.TOKENIZER_SAVE_PATH, exist_ok=True)

print("Конфигурация подпроекта создана ✅")
print(f"Папка для данных: {Config.DATA_DIR}")
print(f"Папка для модели: {Config.MODEL_SAVE_PATH}")
print(f"Папка для токенайзера: {Config.TOKENIZER_SAVE_PATH}")

Конфигурация подпроекта создана ✅
Папка для данных: ./data
Папка для модели: ./llm_model
Папка для токенайзера: ./tokenizer


In [5]:
# 3: 
# Загрузка, очистка, подготовка данных

class DataProcessor:
    def __init__(self):
        self.config = Config()
    
    def load_texts(self) -> List[str]:
        """Загрузка всех .txt файлов"""
        texts = []
        for filename in os.listdir(self.config.DATA_DIR):
            if filename.endswith('.txt'):
                filepath = os.path.join(self.config.DATA_DIR, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        content = f.read().strip()
                        if content:
                            texts.append(content)
                            print(f"- Загружен: {filename} ({len(content)} символов)")
                except Exception as e:
                    print(f"❌ Ошибка загрузки {filename}: {e}")
        return texts
    
    def clean_text(self, text: str) -> str:
        """Очистка текста"""
    
        # Убираем переносы строк
        text = text.replace('\n', ' ').replace('\r', ' ')
    
        # Удаляем все некириллические символы, но сохраняем ВСЕ кириллические буквы (и заглавные и строчные)
        # Сохраняем: кириллические буквы (все регистры), пробелы, русскую пунктуацию
        text = re.sub(r'[^а-яёА-ЯЁ\s\.,!?;:—\-()«»]', '', text)
    
        # Обрабатываем повторяющиеся знаки препинания
        text = re.sub(r'[\.]{2,}', '.', text)      # множественные точки
        text = re.sub(r'[,]{2,}', ',', text)       # множественные запятые
        text = re.sub(r'[!]{2,}', '!', text)       # множественные восклицания
        text = re.sub(r'[?]{2,}', '?', text)       # множественные вопросы
        text = re.sub(r'[;]{2,}', ';', text)       # множественные точки с запятой
        text = re.sub(r'[:]{2,}', ':', text)       # множественные двоеточия
        text = re.sub(r'[-]{2,}', '-', text)       # множественные дефисы/тире
        text = re.sub(r'[—]{2,}', '—', text)       # множественные длинные тире
    
        # Обрабатываем комбинации знаков препинания
        text = re.sub(r'[!?]+', '!', text)         # множественные ! и ? в любых комбинациях
        text = re.sub(r'[\.!?]+', '.', text)       # любые комбинации точек, восклицаний, вопросов
    
        # Нормализуем пробелы (убираем дубли пробелов)
        text = re.sub(r'\s+', ' ', text).strip()
    
        # Переводим в нижний регистр - ВСЕ буквы становятся строчными
        text = text.lower()
    
        # Убираем пробелы перед знаками препинания
        text = re.sub(r'\s+([\.,!?;:])', r'\1', text)
    
        # Добавляем пробелы после знаков препинания, если их нет
        text = re.sub(r'([\.,!?;:])([а-яё])', r'\1 \2', text)
    
        return text
    
    def remove_duplicates(self, texts: List[str]) -> List[str]:
        """Удаление дубликатов"""
        seen = set()
        unique_texts = []
        for text in texts:
            text_hash = hash(text[:1000])
            if text_hash not in seen:
                seen.add(text_hash)
                unique_texts.append(text)
        return unique_texts
    
    def process_data(self) -> List[str]:
        """Основной метод загрузки и очистки данных"""
        print("🔄 Загрузка художественных произведений ...")
        texts = self.load_texts()
        print(f"✅ Загружено {len(texts)} текстов")
        
        if not texts:
            raise ValueError("❌ Не найдено в папке data/")
        
        print("\n🔄Удаление дубликатов...")
        texts = self.remove_duplicates(texts)
        print(f"✅ После удаления дубликатов: {len(texts)} текстов")
        
        print("\n🔄 Очистка текстов...")
        cleaned_texts = []
        for i, text in enumerate(texts):
            cleaned_text = self.clean_text(text)
            if cleaned_text:
                cleaned_texts.append(cleaned_text)
            if (i + 1) % 107 == 0:
                print(f"✅ Обработано {i + 1}/{len(texts)} текстов")
                print(f"После очистки: {len(cleaned_texts)} текстов")
        
        # Статистика по длине текстов
        lengths = [len(text) for text in cleaned_texts]
        print(f" \nСредняя длина текста: {np.mean(lengths):.0f} символов")
        print(f" Минимальная длина: {min(lengths)} символов")
        print(f" Максимальная длина: {max(lengths)} символов")
        
        return cleaned_texts

# Запуск обработки данных
processor = DataProcessor()
cleaned_texts = processor.process_data()

🔄 Загрузка художественных произведений ...
- Загружен: Pushkin_CapitanskaaDochka.txt (214409 символов)
- Загружен: Sologub_KorolevaOrtruda.txt (557621 символов)
- Загружен: Chekhov_Dama.txt (335586 символов)
- Загружен: Gogol_Viy.txt (75506 символов)
- Загружен: Gorky_ZyznKlimaSamgina3.txt (692089 символов)
- Загружен: Zhukova_Dacha.txt (166564 символов)
- Загружен: Nabokov_Otchayanie_1934.txt (276782 символов)
- Загружен: NKhvoshchinskaya_PervayaBorba.txt (359727 символов)
- Загружен: Nabokov_Veschi_1972_Ilin.txt (165577 символов)
- Загружен: Sologub_TjazolyjeSny.txt (640729 символов)
- Загружен: Tolstoi_SemeynoyeSchastiye.txt (165071 символов)
- Загружен: Bulgakov_BelayaGvardiya.txt (504043 символов)
- Загружен: Nabokov_Zashchita_1930.txt (348927 символов)
- Загружен: Gippius_Roman-Tzarevich.txt (426568 символов)
- Загружен: Panaeva_Paseka.txt (169231 символов)
- Загружен: Dostoyevsky_Karamazow1.txt (378137 символов)
- Загружен: Gorky_FomaGordeev.txt (502033 символов)
- Загружен: NKh

In [6]:

# Пример очищенного текста
if cleaned_texts:
    print("\n Пример очищенного текста:")
    print("-" * 55)
    print(cleaned_texts[55][:150] + "..." if len(cleaned_texts[0]) > 150 else cleaned_texts[55])


 Пример очищенного текста:
-------------------------------------------------------
декабря. сегодня я проснулась очень рано. горела свеча на моем столике, у моей постели. на коленях стояла вера и, уткнувшись в мой матрац, плакала. я ...


In [7]:
#  4 : вариант 2
# Обучение токенизатора с WordPiece-style алгоритмом 

from tokenizers import Tokenizer
from tokenizers.models import WordPiece  
from tokenizers.trainers import WordPieceTrainer  
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import WordPiece as WordPieceDecoder  
from transformers import PreTrainedTokenizerFast
import json

class WordPieceTokenizerTrainer:
    def __init__(self):
        self.config = Config()
    
    def train_tokenizer(self, texts: List[str]):
        """Обучение токенизатора с WordPiece-style алгоритмом"""
        print("🔄 Обучение WordPiece-style токенизатора...")
        
        # Используем WordPiece модель
        tokenizer = Tokenizer(WordPiece(
            unk_token="<unk>",
            # префикс "##" для продолжений слов
            continuing_subword_prefix="##"  
        ))
        
        # Пре-токенизатор - разделение по пробелам
        tokenizer.pre_tokenizer = Whitespace()
        
        # WordPiece декодер - знает про "##"
        tokenizer.decoder = WordPieceDecoder()
        
        # Тренер для WordPiece
        trainer = WordPieceTrainer(
            vocab_size=self.config.VOCAB_SIZE,
            min_frequency=2,
            special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"],
            continuing_subword_prefix="##"
            #max_piece_length=16,  # максимальная длина токена
        )
        
        # Обучаем на текстах
        tokenizer.train_from_iterator(texts, trainer=trainer)
        
        # Сохраняем токенизатор
        os.makedirs(self.config.TOKENIZER_SAVE_PATH, exist_ok=True)
        tokenizer.save(os.path.join(self.config.TOKENIZER_SAVE_PATH, "tokenizer.json"))
        
        # Загружаем как transformers tokenizer
        fast_tokenizer = PreTrainedTokenizerFast(
            tokenizer_file=os.path.join(self.config.TOKENIZER_SAVE_PATH, "tokenizer.json"),
            bos_token="<s>",
            eos_token="</s>",
            unk_token="<unk>",
            pad_token="<pad>",
            mask_token="<mask>",
        )
        
        print(f"✅ WordPiece токенизатор обучен и сохранен")
        print(f"--> Размер словаря: {fast_tokenizer.vocab_size}")
        
        # Проверяем созданные файлы
        print(f"\n Созданные файлы в {self.config.TOKENIZER_SAVE_PATH}:")
        for file in os.listdir(self.config.TOKENIZER_SAVE_PATH):
            print(f"   - {file}")
        
        # Пример токенизации
        test_text = "Высокохудожественный пример текста для токенизации, предобучения большой языковой модели."
        test_text=test_text.lower()
        tokens = fast_tokenizer.tokenize(test_text)
        print(f"\n Пример токенизации: '{test_text}'")
        print(f"   -> Токены: {tokens}")
        
        # Декодирование обратно
        encoded = fast_tokenizer.encode(test_text)
        decoded = fast_tokenizer.decode(encoded)
        print(f"   -> Закодировано: {encoded}")
        print(f"   -> Декодировано: '{decoded}'")
        
        return fast_tokenizer
    
    def comprehensive_test(self, tokenizer):
        """Тестирование WordPiece токенизатора"""
        test_texts = [
            "высокохудожественный пример текста",
            "предобучения большой языковой модели",
            "машина обучения работает прекрасно", 
            "это тестирование декодирования слов",
            "привет мир как дела"
        ]
        
        print(f"\n🎯 ТЕСТИРОВАНИЕ WordPiece:")
        
        for i, test_text in enumerate(test_texts, 1):
            # Токенизация
            tokens = tokenizer.tokenize(test_text)
            
            # Кодирование и декодирование
            encoded = tokenizer.encode(test_text)
            decoded = tokenizer.decode(encoded)
            
            print(f"\n{i}. Оригинал: '{test_text}'")
            print(f"   Токены: {tokens}")
            print(f"   Декодировано: '{decoded}'")
            
            # Анализ токенов
            self.analyze_tokens(tokens)
            
            if decoded == test_text:
                print(f"   ✅ ПРОЙДЕНО")
            else:
                print(f"   ❌ НЕ ПРОЙДЕНО")
    
    def analyze_tokens(self, tokens):
        """Анализ структуры токенов"""
        print(f"   📊 Анализ токенов:")
        current_word = []
        
        for token in tokens:
            if token.startswith("##"):
                # Продолжение слова
                current_word.append(token[2:])  # убираем "##"
                print(f"      '{token}' → продолжение слова")
            else:
                # Начало нового слова
                if current_word:
                    print(f"      Слово: '{''.join(current_word)}'")
                current_word = [token]
                print(f"      '{token}' → начало слова")
        
        if current_word:
            print(f"      Слово: '{''.join(current_word)}'")

# Обучение токенизатора
tokenizer_trainer = WordPieceTokenizerTrainer()
tokenizer = tokenizer_trainer.train_tokenizer(cleaned_texts)

🔄 Обучение WordPiece-style токенизатора...



✅ WordPiece токенизатор обучен и сохранен
--> Размер словаря: 3000

 Созданные файлы в ./tokenizer:
   - tokenizer.json

 Пример токенизации: 'высокохудожественный пример текста для токенизации, предобучения большой языковой модели.'
   -> Токены: ['высо', '##ко', '##ху', '##до', '##же', '##ствен', '##ный', 'при', '##мер', 'те', '##к', '##ста', 'для', 'то', '##ке', '##низ', '##а', '##ции', ',', 'пред', '##об', '##у', '##чен', '##и', '##я', 'большой', 'язы', '##ково', '##й', 'мо', '##дели', '.']
   -> Закодировано: [993, 100, 1150, 137, 184, 910, 223, 143, 440, 728, 52, 186, 385, 126, 313, 1950, 56, 1189, 7, 366, 393, 63, 970, 48, 72, 2205, 1879, 2917, 65, 159, 1891, 9]
   -> Декодировано: 'высокохудожественный пример текста для токенизации, предобучения большой языковой модели.'


In [8]:
# 5: 
# Создание датасета

class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, texts: List[str]):
        self.tokenizer = tokenizer
        self.config = Config()
        self.encodings = self._process_texts(texts)
    
    def _process_texts(self, texts: List[str]) -> Dict[str, torch.Tensor]:
        """Токенизация и разбиение на чанки"""
        all_tokens = []
        
        print("\n🔄 Токенизация и разбиение на чанки...")
        total_chunks = 0
        
        for text_idx, text in enumerate(texts):
            # Токенизируем текст
            tokens = self.tokenizer.encode(
                text, 
                add_special_tokens=False,
                truncation=False
            )
            
            chunk_size = self.config.CONTEXT_LENGTH - 2  # 512-2 где 2-место для BOS и EOS
            
            for i in range(0, len(tokens), chunk_size):
                chunk = tokens[i:i + chunk_size]
                if len(chunk) >= 32:  # минимальный размер чанка
                    # Добавляем BOS и EOS
                    chunk_with_special = [self.tokenizer.bos_token_id] + chunk + [self.tokenizer.eos_token_id]
                    # Дополняем до нужной длины
                    if len(chunk_with_special) < self.config.CONTEXT_LENGTH:
                        chunk_with_special.extend([self.tokenizer.pad_token_id] * 
                                                (self.config.CONTEXT_LENGTH - len(chunk_with_special)))
                    all_tokens.append(chunk_with_special[:self.config.CONTEXT_LENGTH])
                    total_chunks += 1
            
            if (text_idx + 1) % 50 ==0 or (text_idx + 1) % 107 == 0:
                print(f"\n -- Обработано {text_idx + 1}/{len(texts)} текстов, создано {total_chunks} чанков")
        
        input_ids = torch.tensor(all_tokens, dtype=torch.long)
        
        print(f"\n✅ Создано {len(all_tokens)} примеров для обучения")
        print(f" Размерность данных: {input_ids.shape}")
        
        return {
            'input_ids': input_ids,
            'labels': input_ids.clone()
        }
    
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'labels': self.encodings['labels'][idx]
        }

# Создание датасета
print("Подготовка датасета...")
print("-" * 55)
dataset = TextDataset(tokenizer, cleaned_texts)
print(f"\n✅ Размер датасета: {len(dataset)} примеров")

Подготовка датасета...
-------------------------------------------------------

🔄 Токенизация и разбиение на чанки...

 -- Обработано 50/107 текстов, создано 13072 чанков

 -- Обработано 100/107 текстов, создано 25417 чанков

 -- Обработано 107/107 текстов, создано 27303 чанков

✅ Создано 27303 примеров для обучения
 Размерность данных: torch.Size([27303, 512])

✅ Размер датасета: 27303 примеров


In [9]:
#  6: 
# Создание модели и коллбэков

def create_model(tokenizer: PreTrainedTokenizer):
    """Создание модели"""
    print("Создание модели...")
    print("-" * 55)
    
    
    config = LlamaConfig(
        vocab_size=tokenizer.vocab_size,
        hidden_size=Config.HIDDEN_SIZE,
        intermediate_size=Config.INTERMEDIATE_SIZE,
        num_hidden_layers=Config.NUM_HIDDEN_LAYERS,
        num_attention_heads=Config.NUM_ATTENTION_HEADS,
        num_key_value_heads=Config.NUM_KEY_VALUE_HEADS,
        max_position_embeddings=Config.CONTEXT_LENGTH,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        tie_word_embeddings=False,
    )
    
    model = LlamaForCausalLM(config)
    
    # Подсчет параметров
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n✅ Модель создана")
    print(f"-- Общее количество параметров: {total_params:,}")
    #print(f"-- Примерно {total_params/1e6:.1f}M параметров")
    
    return model

# Коллбэк для валидации
class ValidationCallback(TrainerCallback):
    def __init__(self, tokenizer, test_prompts, generation_length=50):
        self.tokenizer = tokenizer
        self.test_prompts = test_prompts
        self.generation_length = generation_length
        self.losses = []
        self.perplexities = []
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        """Логирование прогресса"""
        if logs and 'loss' in logs:
            loss = logs['loss']
            perplexity = torch.exp(torch.tensor(loss)).item()
            self.losses.append(loss)
            self.perplexities.append(perplexity)
            
            if state.global_step % 100 == 0:
                print(f"-> Шаг {state.global_step}: Loss = {loss:.4f}, Perplexity = {perplexity:.4f}")
    
    def on_evaluate(self, args, state, control, model, **kwargs):
        """Генерация текста на тестовых промптах"""
        if state.global_step % 500 == 0:
            print(f"\n{'='*55}")
            print(f"Генерация на шаге {state.global_step}:")
            print(f"{'='*55}")
            
            model.eval()
            with torch.no_grad():
                for i, prompt in enumerate(self.test_prompts):
                    inputs = self.tokenizer.encode(prompt, return_tensors="pt")
                    
                    if torch.cuda.is_available():
                        inputs = inputs.cuda()
                        model = model.cuda()
                    
                    outputs = model.generate(
                        inputs,
                        max_new_tokens=self.generation_length,
                        do_sample=True,
                        temperature=0.8,
                        top_p=0.9,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )
                    
                    generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                    print(f"\n Промпт {i+1}: {prompt}")
                    print(f"--> Сгенерировано: {generated_text}")
                    print("-" * 55)
            
            model.train()

# Создание модели
model = create_model(tokenizer)

# Инициализация коллбэка
validation_callback = ValidationCallback(tokenizer, Config.TEST_PROMPTS)

Создание модели...
-------------------------------------------------------

✅ Модель создана
-- Общее количество параметров: 132,006,912


In [10]:
#  7 : 
# Настройка обучения


def get_training_arguments():
    """Возвращает аргументы обучения"""
    
    if torch.cuda.is_available():
        print("--> Обучение на GPU")
        return TrainingArguments(
            output_dir="./gpu",
            overwrite_output_dir=True,
            per_device_train_batch_size=2,
            gradient_accumulation_steps=8,
            learning_rate=5e-4,
            weight_decay=0.1,
            max_steps=1000,
            warmup_steps=50,
            logging_steps=50,
            save_steps=500,
            eval_steps=250,
            prediction_loss_only=True,
            remove_unused_columns=False,
            fp16=True,
            dataloader_pin_memory=False,
            report_to=None,
        )
    else:
        print("--> Обучение на CPU")
        return TrainingArguments(
            output_dir="./cpu",
            overwrite_output_dir=True,
            per_device_train_batch_size=2,
            gradient_accumulation_steps=8,  # эффективный batch 16
            learning_rate=5e-4,
            weight_decay=0.1,
            max_steps=1000,  # для демонстрации на CPU
            warmup_steps=50,
            logging_steps=25,
            save_steps=500,
            prediction_loss_only=True,
            remove_unused_columns=False,
            dataloader_pin_memory=False,
            report_to=None,
        )

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Получение аргументов обучения
training_args = get_training_arguments()

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    processing_class=tokenizer, # изменение tokenizer=tokenizer
    callbacks=[validation_callback],
)

print("✅ Тренер инициализирован")
print(f"-- Эффективный batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"-- Всего шагов: {training_args.max_steps}")
print(f"-- Learning rate: {training_args.learning_rate}")
print(f"-- Weight decay: {training_args.weight_decay}")


--> Обучение на GPU
✅ Тренер инициализирован
-- Эффективный batch size: 16
-- Всего шагов: 1000
-- Learning rate: 0.0005
-- Weight decay: 0.1


In [11]:
#  8 : 
# Запуск обучения

print("🔄 Обучени модели ...")
print("=" * 55)

# Запуск обучения
trainer.train()

print("\n✅ Обучение завершено!")

🔄 Обучени модели ...


Step,Training Loss
50,6.860500
100,6.146800
150,5.760000
200,5.448700
250,5.194400
300,4.974800
350,4.814700
400,4.672300
450,4.561700
500,4.464700


-> Шаг 100: Loss = 6.1468, Perplexity = 467.2199
-> Шаг 200: Loss = 5.4487, Perplexity = 232.4558
-> Шаг 300: Loss = 4.9748, Perplexity = 144.7199
-> Шаг 400: Loss = 4.6723, Perplexity = 106.9434
-> Шаг 500: Loss = 4.4647, Perplexity = 86.8950
-> Шаг 600: Loss = 4.2833, Perplexity = 72.4792
-> Шаг 700: Loss = 4.1443, Perplexity = 63.0735
-> Шаг 800: Loss = 4.0408, Perplexity = 56.8718
-> Шаг 900: Loss = 3.9425, Perplexity = 51.5473
-> Шаг 1000: Loss = 3.9042, Perplexity = 49.6104

✅ Обучение завершено!


In [12]:
#  9 : 
# Сохранение модели и финальная оценка

print("🔄 Сохранение модели...")
trainer.save_model(Config.MODEL_SAVE_PATH)
tokenizer.save_pretrained(Config.MODEL_SAVE_PATH)
print(f"✅ Модель сохранена в {Config.MODEL_SAVE_PATH}")

# Финальная оценка
class QualityEvaluator:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        print(f"🔧 Модель перемещена на: {self.device}")
    
    def evaluate_generation(self, max_length=100):
        """Оценка качества генерации"""
        print("\n Финальная оценка качества:")
        print("=" * 55)
        
        self.model.eval()
        results = []
        
        with torch.no_grad():
            for i, prompt in enumerate(Config.TEST_PROMPTS):
                prompt=prompt.lower()
                inputs = self.tokenizer.encode(prompt, return_tensors="pt")
                
                inputs = inputs.to(self.device)
                
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=max_length,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.85,
                    pad_token_id=self.tokenizer.pad_token_id,
                )
                
                generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                print(f"\n Промпт {i+1}: {prompt}")
                print(f"-> Сгенерированный моделью текст: {generated_text}")
                print("-" * 55)
                
                results.append({
                    'prompt': prompt,
                    'generated_text_raw': generated_text
                })
        
        return results

# Загрузка обученной модели для оценки
print("\n🔄 Загрузка обученной модели для оценки...")
trained_model = LlamaForCausalLM.from_pretrained(Config.MODEL_SAVE_PATH)
trained_tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_SAVE_PATH)

evaluator = QualityEvaluator(trained_tokenizer, trained_model)
final_results = evaluator.evaluate_generation()

# Сохранение результатов
with open(f"{Config.MODEL_SAVE_PATH}/final_results.json", "w", encoding="utf-8") as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ Результаты сохранены в {Config.MODEL_SAVE_PATH}/final_results.json")

🔄 Сохранение модели...
✅ Модель сохранена в ./llm_model

🔄 Загрузка обученной модели для оценки...
🔧 Модель перемещена на: cuda

 Финальная оценка качества:

 Промпт 1: все мысли, которые имеют огромные последствия
-> Сгенерированный моделью текст: все мысли, которые имеют огромные последствия и были. они и не могли бы жить и, но не в силах, в которых он, не только не мог, как это было бы не только для него, но в самом деле, в то время как он чувствовал, и он был очень легко. он не мог себе представить, что он не знал, как он сделал. но когда же он был в нем, он чувствовал, что он в петербурге, когда он, несмотря на то, что он не знал, он чувствовал
-------------------------------------------------------

 Промпт 2: сила войска зависит от его духа
-> Сгенерированный моделью текст: сила войска зависит от его духа, а за границу и на вам - то и теперь и не пропустило, а он и не до сих пор не мог, а только на чтошне не в чем - то - то, - но в таком - то не таком, как и что, как бы в таком 